In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

### Generate synthetic data

In [ ]:
from synthetics import generate_single_task_unipolar, gaussian_bags_of_words, vocab1k

N = 10000
M = 30
K = 2
NUM_SPLITS = 3

Ds = [] # data
Xs = [] # features
Ls = [] # noisy labels
Ys = [] # true labels
for _ in range(NUM_SPLITS):
    L, Y, _ = generate_single_task_unipolar(
        N, M, k=K, alpha_range=[0.6, 0.9], beta_range=[0.1, 0.2], 
        class_balance=[0.3, 0.7], seed=1)
    
    X, D = gaussian_bags_of_words(Y, vocab1k)
    
    Ls.append(L)
    Ys.append(Y)
    Ds.append(D)
    Xs.append(X)

### Apply LabelModel

In [ ]:
from metal.label_model import LabelModel

lm = LabelModel(seed=2)
lm.train(Ls[0], L_dev=Ls[1], Y_dev=Ys[1], n_epochs=1000, lr=0.01, print_at=100)
Y_p = lm.score(Ls[1], Ys[1])

In [ ]:
Y_p = lm.predict(Ls[1])

### Random Search

In [ ]:
search_space = {
    'print_at': 200,
    'n_epochs': 1000, # a single constant value
    'l2': [0, 0.1, 0.01], # a list of discrete values
    'lr': {'range': [0.01, 0.0001], 'scale': 'log'}, # a range and scale to interpolate by
}

In [ ]:
from metal.tuner import ModelTuner

tuner = ModelTuner(LabelModel)
init_args = []
train_args = [Ls[0]]
model, best_config = tuner.search(init_args, train_args, Ls[1], Ys[1], 
                                  search_space, max_search=5, metric='accuracy')

### Metrics

Calculate metrics in one of two ways.

1. Use metric_score() and pass the metric name
3. The the specific metric's function (e.g., accuracy_score())

In [ ]:
from metal.metrics import metric_score, accuracy_score

metric_score(Ys[1], Y_p, 'accuracy')
accuracy_score(Ys[1], Y_p)

Built-in metrics include:

In [ ]:
metric_score(Ys[1], Y_p, 'accuracy')
metric_score(Ys[1], Y_p, 'coverage')
metric_score(Ys[1], Y_p, 'precision')
metric_score(Ys[1], Y_p, 'recall')
metric_score(Ys[1], Y_p, 'f1')
metric_score(Ys[1], Y_p, 'fbeta', beta=1.0)

### Confusion matrix

In [ ]:
from metal.analysis import confusion_matrix
confusion_matrix(Y_p, Ys[1])

In [ ]:
confusion_matrix(Y_p, Ys[1], normalize=True)

In [ ]:
mat = confusion_matrix(Y_p, Ys[1], pretty_print=True)

### Error analysis

In [ ]:
from metal.analysis import error_buckets

buckets = error_buckets(Y_p, Ys[1], Ds[1])
tp = buckets[1,1]

In [ ]:
tp[:3]

### Label matrix analysis

In [ ]:
from metal.analysis import (
    item_coverage,
    item_overlap,
    item_conflict,
    LF_accuracies,
    LF_coverages,
    LF_overlaps,
    LF_conflicts,
)

In [ ]:
item_coverage(Ls[0])

In [ ]:
item_overlap(Ls[0])

In [ ]:
item_conflict(Ls[0])

In [ ]:
LF_accuracies(Ls[0], Ys[0])

In [ ]:
LF_coverages(Ls[0])

In [ ]:
LF_overlaps(Ls[0])

In [ ]:
LF_conflicts(Ls[0])

In [ ]:
from metal.analysis import (
    view_label_matrix,
    view_overlaps,
    view_conflicts,
)

view_label_matrix(Ls[0])

In [ ]:
view_overlaps(Ls[0], normalize=True)

In [ ]:
view_conflicts(Ls[0], normalize=False)